In [ ]:
#importing libraries and packages
import sys,os,time
print(sys.executable)
import pandas as pd
#from datetime import datetime
from datetime import datetime, timedelta
import numpy as np
#import matplotlib as mplt
import matplotlib.pyplot as plt
import matplotlib.markers
from pandas.tseries.frequencies import to_offset# needed to set the time offset to match to the muons DF
import seaborn as sns
from numpy import argmax
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.datasets import make_regression
from sklearn.preprocessing import LabelEncoder
#from tensorflow.keras import Sequential
#from tensorflow.keras.layers import Dense
#import tensorflow as tf
#import pylab as plt
#import ROOT

In [ ]:
#pd.Timestamp('2019-10-21T12:05:04.126Z')
dtypes = {'date': 'Timestamp'}
fname1 = '/home/cosmic/onlineMon/data/Muon000/HeHome2020-11-03-to-2021-06-28.log'
fname2 = '/home/cosmic/onlineMon/data/Muon000/Muon000-2022-04-23-to-2022-05-15.log'
fname3 = '/home/cosmic/onlineMon/data/Muon000/muon000.log'
#df1 = pd.read_csv(fname3,index_col="date",parse_dates=['date'])
dft1  = pd.read_csv(fname1,index_col="date",parse_dates=['date'])
dft2  = pd.read_csv(fname2,index_col="date",parse_dates=['date'])
dft3  = pd.read_csv(fname2,index_col="date",parse_dates=['date'])

frames = [dft1, dft2,dft3]
df1  = pd.concat(frames)

df1.rename(columns = {'One and Three':'Ch13', 'One and Two':'Ch12', 'Two and Three':'Ch23'},inplace = True)
df1.drop(df1.columns[[0,1,2,6,10]],axis=1, inplace=True)
df1.dtypes
#df1

In [ ]:
df1.info
df1

In [ ]:
#add_time= datetime.timedelta(hours=5)
#df1['date'] = df1['date'] + timedelta(hours=5)
df1.index=pd.to_datetime(df1.index)-timedelta(hours=5)
df1

In [ ]:
df1_hourly = df1.resample('H').sum()

df1_hourly.drop(df1_hourly.head(1).index,inplace=True)
df1_hourly.drop(df1_hourly.tail(1).index,inplace=True)
df1_hourly.drop(df1_hourly[df1_hourly['Ch12'] < 7000].index, inplace = True)
pd.to_datetime(df1_hourly.index).hour
pd.to_datetime(df1_hourly.index).hour
df1_hourly.head(10)
df1_hourly
#df1_hourly[df1_hourly['Ch12'] <7000]
df1_hourly.drop(df1_hourly.head(1).index,inplace=True)
df1_hourly.drop(df1_hourly.tail(1).index,inplace=True)
#df1_hourly.drop(df1_hourly[df1_hourly['Ch12'] < 7000].index, inplace = True)
pd.to_datetime(df1_hourly.index).hour
df1_hourly.head(10)
df1_hourly

# Download the Pressure Temperature file

In [ ]:
fPT1    ='mslp_temp.csv'
df1_PT  = pd.read_csv(fPT1)
df1_PT.drop(df1_PT[df1_PT['mslp'] == 'M'].index, inplace = True)
df1_PT.drop(df1_PT[df1_PT['tmpf'] == 'M'].index, inplace = True)
df1_PT.dropna(inplace=True)
df1_PT['valid'] = pd.to_datetime(df1_PT['valid'], utc =True)
df1_PT = df1_PT.set_index('valid')
df1_PT = df1_PT.drop(columns=["station"])
loffset = '-52min'
df1_PT.index = df1_PT.index + to_offset(loffset)
df1_PT["mslp"] = df1_PT["mslp"].astype(str).astype(float)
df1_PT["tmpf"] = df1_PT["tmpf"].astype(str).astype(float)
df1_PT

In [ ]:
merged_df1_all=pd.merge(df1_hourly,df1_PT, how='inner', left_index=True, right_index=True)
merged_df1_all

In [ ]:
fig, axs = plt.subplots(2, 3, sharey=False, sharex=False,tight_layout=True,figsize=(12, 8))
#fig.suptitle('Muon001)')
axs[0,0].hist(merged_df1_all['One'], bins=80,histtype='step',label='One(Muon000)',color='blue')
axs[0,0].set_title("Layer One(Muon000)")
#axs[0,0].set_xlim(900,1400)
axs[0,1].hist(merged_df1_all['Two'], bins=80,histtype='step',label='Two(Muon000)',color='blue')
axs[0,1].set_title("Layer Two(Muon000)")
#axs[0,1].set_xlim(1000,1500)
axs[0,2].hist(merged_df1_all['Three'], bins=80,histtype='step',label='Three(Muon000)',color='blue')
axs[0,2].set_title("Layer Three(Muon000)")
#axs[0,2].set_xlim(900,1500)
axs[1,0].hist(merged_df1_all['Ch12'],histtype='step',bins=100,color='blue')
axs[1,0].set_title("Muon000 Coincidence (1&2)")
axs[1,1].hist(merged_df1_all['Ch23'],histtype='step',bins=100,color='blue')
axs[1,1].set_title("Muon000 Coincidence (2&3)")
axs[1,2].hist(merged_df1_all['Ch13'],histtype='step',bins=100,color='blue')
axs[1,2].set_title("Muon000 Coincidence (1&3)")

#axs[1,2].set_xlim(900,1500)
#plt.xticks(fontsize=14)
#plt.yticks(fontsize=14)
#plt.legend(['Example legend'],loc='upper center')
plt.savefig('/home/cosmic/onlineMon/images/M000_ch_coinc_mostrecent.png')

In [ ]:
merged_df1_all = merged_df1_all.drop(columns=["One","Two","Three"])

In [ ]:
from datetime import datetime, timedelta
now = datetime.utcnow().date()
yesterday = datetime.utcnow().date() - timedelta(days=1)
lastmonth = datetime.utcnow().date() - timedelta(days=30)
print(now)
print(yesterday)
print(lastmonth)
df1_tmp1 = (merged_df1_all.loc['2020-9-14':'2021-5-14']).copy()
df1_tmp2 = (merged_df1_all.loc['2021-09-14':now]).copy()
#framesnew = [df1_tmp1, df1_tmp2]
#merged_df1  = df1_tmp2.copy()#pd.concat(framesnew)

In [ ]:
merged_df1_1day   = merged_df1_all.loc[yesterday:now].copy()
merged_df1_1day
merged_df1_31days = merged_df1_all.loc[lastmonth:now].copy() 

In [ ]:
def calc_percent_change(merged_df):
    merged_df["Ch12_pct"] = ((merged_df['Ch12'] - merged_df['Ch12'].mean())/ 
                      merged_df['Ch12'].mean()*100)
    merged_df["Ch13_pct"] = ((merged_df['Ch13'] - merged_df['Ch13'].mean())/ 
                      merged_df['Ch13'].mean()*100)
    merged_df["Ch23_pct"] = ((merged_df['Ch23'] - merged_df['Ch23'].mean())/ 
                      merged_df['Ch23'].mean()*100)
    merged_df["Ch12_m_avg"] = merged_df.rolling('6H').Ch12_pct.mean()
    merged_df["Ch13_m_avg"] = merged_df.rolling('6H').Ch13_pct.mean()
    merged_df["Ch23_m_avg"] = merged_df.rolling('6H').Ch23_pct.mean()
    merged_df["press_pct"] = ((merged_df['mslp'] - merged_df['mslp'].mean())/ 
                      merged_df['mslp'].mean()*100)
    merged_df["temp_pct"] = ((merged_df['tmpf'] - merged_df['tmpf'].mean())/ 
                      merged_df['tmpf'].mean()*100)
    merged_df['temp_scaled'] = 0.1*merged_df['temp_pct']
    merged_df["delta_Ch12"]  = np.log((merged_df["Ch12"])/merged_df["Ch12"].mean())
    merged_df["delta_Ch13"]  = np.log((merged_df["Ch13"])/merged_df["Ch13"].mean())
    merged_df["delta_Ch23"]  = np.log((merged_df["Ch23"])/merged_df["Ch23"].mean())
    merged_df["delta_pres"] = (merged_df["mslp"]-merged_df["mslp"].mean())
    print(merged_df["mslp"].mean(),merged_df["mslp"].std(), merged_df['Ch12_pct'].max()*1.2, merged_df["Ch12"].mean(),merged_df["Ch12_pct"].std())
    

## Compute the %age change and rolling average¶

In [ ]:
#calc_percent_change(merged_df1_1day)
#calc_percent_change(merged_df1_31days)
#calc_percent_change(merged_df1_all)

In [ ]:
merged_df1_1day["Ch12_pct"] = ((merged_df1_1day['Ch12'] - merged_df1_1day['Ch12'].mean())/ 
                      merged_df1_1day['Ch12'].mean()*100)
merged_df1_1day["Ch13_pct"] = ((merged_df1_1day['Ch13'] - merged_df1_1day['Ch13'].mean())/ 
                      merged_df1_1day['Ch13'].mean()*100)
merged_df1_1day["Ch23_pct"] = ((merged_df1_1day['Ch23'] - merged_df1_1day['Ch23'].mean())/ 
                      merged_df1_1day['Ch23'].mean()*100)
merged_df1_1day["Ch12_m_avg"] = merged_df1_1day.rolling('6H').Ch12_pct.mean()
merged_df1_1day["Ch13_m_avg"] = merged_df1_1day.rolling('6H').Ch13_pct.mean()
merged_df1_1day["Ch23_m_avg"] = merged_df1_1day.rolling('6H').Ch23_pct.mean()
merged_df1_1day["press_pct"] = ((merged_df1_1day['mslp'] - merged_df1_1day['mslp'].mean())/ 
                      merged_df1_1day['mslp'].mean()*100)
merged_df1_1day["temp_pct"] = ((merged_df1_1day['tmpf'] - merged_df1_1day['tmpf'].mean())/ 
                      merged_df1_1day['tmpf'].mean()*100)
merged_df1_1day['temp_scaled'] = 0.1*merged_df1_1day['temp_pct']

In [ ]:
merged_df1_31days["Ch12_pct"] = ((merged_df1_31days['Ch12'] - merged_df1_31days['Ch12'].mean())/ 
                      merged_df1_31days['Ch12'].mean()*100)
merged_df1_31days["Ch13_pct"] = ((merged_df1_31days['Ch13'] - merged_df1_31days['Ch13'].mean())/ 
                      merged_df1_31days['Ch13'].mean()*100)
merged_df1_31days["Ch23_pct"] = ((merged_df1_31days['Ch23'] - merged_df1_31days['Ch23'].mean())/ 
                      merged_df1_31days['Ch23'].mean()*100)
merged_df1_31days["Ch12_m_avg"] = merged_df1_31days.rolling('6H').Ch12_pct.mean()
merged_df1_31days["Ch13_m_avg"] = merged_df1_31days.rolling('6H').Ch13_pct.mean()
merged_df1_31days["Ch23_m_avg"] = merged_df1_31days.rolling('6H').Ch23_pct.mean()
merged_df1_31days["press_pct"] = ((merged_df1_31days['mslp'] - merged_df1_31days['mslp'].mean())/ 
                      merged_df1_31days['mslp'].mean()*100)
merged_df1_31days["temp_pct"] = ((merged_df1_31days['tmpf'] - merged_df1_31days['tmpf'].mean())/ 
                      merged_df1_31days['tmpf'].mean()*100)
merged_df1_31days['temp_scaled'] = 0.1*merged_df1_31days['temp_pct']
merged_df1_31days

In [ ]:
df1_tmp1["Ch12_pct"] = ((df1_tmp1['Ch12'] - df1_tmp1['Ch12'].mean())/ 
                      df1_tmp1['Ch12'].mean()*100)
df1_tmp1["Ch13_pct"] = ((df1_tmp1['Ch13'] - df1_tmp1['Ch13'].mean())/ 
                      df1_tmp1['Ch13'].mean()*100)
df1_tmp1["Ch23_pct"] = ((df1_tmp1['Ch23'] - df1_tmp1['Ch23'].mean())/ 
                      df1_tmp1['Ch23'].mean()*100)

df1_tmp2["Ch12_pct"] = ((df1_tmp2['Ch12'] - df1_tmp2['Ch12'].mean())/ 
                      df1_tmp2['Ch12'].mean()*100)
df1_tmp2["Ch13_pct"] = ((df1_tmp2['Ch13'] - df1_tmp2['Ch13'].mean())/ 
                      df1_tmp2['Ch13'].mean()*100)
df1_tmp2["Ch23_pct"] = ((df1_tmp2['Ch23'] - df1_tmp2['Ch23'].mean())/ 
                      df1_tmp2['Ch23'].mean()*100)

framesnew   = [df1_tmp1, df1_tmp2]
merged_df1  = pd.concat(framesnew)

#merged_df1["Ch12_pct"] = ((merged_df1['Ch12'] - merged_df1['Ch12'].mean())/ 
#                      merged_df1['Ch12'].mean()*100)
#merged_df1["Ch13_pct"] = ((merged_df1['Ch13'] - merged_df1['Ch13'].mean())/ 
#                      merged_df1['Ch13'].mean()*100)
#merged_df1["Ch23_pct"] = ((merged_df1['Ch23'] - merged_df1['Ch23'].mean())/ 
#                      merged_df1['Ch23'].mean()*100)

merged_df1["Ch12_m_avg"] = merged_df1.rolling('6H').Ch12_pct.mean()
merged_df1["Ch13_m_avg"] = merged_df1.rolling('6H').Ch13_pct.mean()
merged_df1["Ch23_m_avg"] = merged_df1.rolling('6H').Ch23_pct.mean()

merged_df1["press_pct"] = ((merged_df1['mslp'] - merged_df1['mslp'].mean())/ 
                      merged_df1['mslp'].mean()*100)
merged_df1["temp_pct"] = ((merged_df1['tmpf'] - merged_df1['tmpf'].mean())/ 
                      merged_df1['tmpf'].mean()*100)
merged_df1['temp_scaled'] = 0.1*merged_df1['temp_pct']
#merged_df1["delta_cts"]  = (merged_df1["counts"]-merged_df1["counts"].mean())/merged_df1["counts"]
#merged_df1["delta_pres"] = (merged_df1["mslp"]-merged_df1["mslp"].mean())/merged_df1["mslp"]
merged_df1["delta_Ch12"]  = np.log((merged_df1["Ch12"])/merged_df1["Ch12"].mean())
merged_df1["delta_Ch13"]  = np.log((merged_df1["Ch13"])/merged_df1["Ch13"].mean())
merged_df1["delta_Ch23"]  = np.log((merged_df1["Ch23"])/merged_df1["Ch23"].mean())
merged_df1["delta_pres"] = (merged_df1["mslp"]-merged_df1["mslp"].mean())
merged_df1
#df1_hourly.describe()

## Plot the %age change

In [ ]:
fig = plt.figure(figsize=(14, 12),constrained_layout=True)
gs = fig.add_gridspec(3)
axs = gs.subplots(sharex=False, sharey=False)
#fig.suptitle('Flux %age Change (Muon001 Detector)')
#axs[0].set_title("Last 24 hours (yesterday:now)")
axs[0].set_title('Last 24 hours:'+str(now)+' to ' +str(yesterday),fontsize=15,color='blue')
axs[0].plot(merged_df1_1day['temp_scaled'],
            linestyle='--', linewidth=0.5, color='gray', label='scaled Temp(0.1) %')
axs[0].plot(merged_df1_1day['Ch12_m_avg'],
            linestyle='-', linewidth=1, color='red', label='Rolling avg(6Hr)Ch12 %')
axs[0].plot(merged_df1_1day["Ch13_m_avg"],
            linestyle='-', linewidth=1, color='green', label="Rolling avg(6Hr)Ch13 %")
axs[0].plot(merged_df1_1day["Ch23_m_avg"],
            linestyle='-', linewidth=1, color='black', label="Rolling avg(6Hr)Ch23 %")
axs[0].plot(merged_df1_1day['press_pct'],
            linestyle='--', linewidth=1, color='blue', label='Press %')


axs[1].set_title("Last 30 days",fontsize=15,color='blue')
axs[1].plot(merged_df1_31days['temp_scaled'],
            linestyle='--', linewidth=0.5, color='gray', label='scaled Temp(0.1) %')
axs[1].plot(merged_df1_31days['Ch12_m_avg'],
            linestyle='--', linewidth=2, color='red', label='Rolling avg(6Hr)Ch12 %')
#axs[1].plot(merged_df1_31days["Ch13_m_avg"],
#marker='*', markersize=3, linestyle='-', linewidth=0.5, color='green', label="Rolling avg(6Hr)Ch13 %")
#axs[1].plot(merged_df1_31days["Ch23_m_avg"],
#marker='v', markersize=3, linestyle='-', linewidth=0.5, color='blue', label="Rolling avg(6Hr)Ch23 %")
axs[1].plot(merged_df1_31days['press_pct'],
            linestyle='--', linewidth=1, color='blue', label='Press %')


axs[2].set_title("All data",fontsize=15,color='blue')
axs[2].plot(merged_df1['temp_scaled'],
            linestyle='--', linewidth=0.5, color='gray', label='scaled Temp(0.1) %')
#axs[2].plot(merged_df1["Ch13_m_avg"],
#marker='*', markersize=3, linestyle='-', linewidth=0.5, color='green', label="Rolling avg(6Hr)Ch13 %")
#axs[2].plot(merged_df1["Ch23_m_avg"],        
#marker='v', markersize=3, linestyle='-', linewidth=0.5, color='blue', label="Rolling avg(6Hr)Ch23 %")
axs[2].plot(merged_df1["Ch12_m_avg"],
            linestyle='--', linewidth=2, color='red', label='Rolling avg(6Hr)Ch12 %')
axs[2].plot(merged_df1['press_pct'],
            linestyle='--', linewidth=1, color='blue', label='Press %')

#axs[2].plot(df1_hourly["Ch23_m_avg"],
#marker='.', markersize=1, linestyle='-', linewidth=1, color='black', label='Ch23_m_avg')

axs[0].set_ylabel('Flux %age Change',fontsize=15)
axs[0].set_ylim([-5, 5])
axs[0].legend();
axs[0].legend(ncol=3)
axs[0].legend(ncol=3,frameon=False,fontsize=15)
axs[0].tick_params(axis='x', labelsize=13)
axs[0].grid(True, color = 'DarkTurquoise', alpha=1, linestyle = '--', linewidth = 0.5)

axs[1].set_ylabel('Flux %age Change',fontsize=15)
axs[1].set_ylim([-7, 7])
axs[1].legend();
axs[1].legend(ncol=3)
axs[1].legend(ncol=3,frameon=False,fontsize=15)
axs[1].tick_params(axis='x', labelsize=13)
axs[1].grid(True, color = 'DarkTurquoise', alpha=1, linestyle = '--', linewidth = 0.5)

axs[2].set_ylabel('Flux %age Change',fontsize=15)
axs[2].set_ylim([-8, 8])
axs[2].legend();
axs[2].legend(ncol=3)
axs[2].legend(ncol=3,frameon=False,fontsize=15)
axs[2].tick_params(axis='x', labelsize=13)
axs[2].grid(True, color = 'DarkTurquoise', alpha=1, linestyle = '--', linewidth = 0.5)
# Hide x labels and tick labels for all but bottom plot.
#for ax in axs:
 #   ax.label_outer()
plt.savefig('/home/cosmic/onlineMon/images/M000_pct_Ch_mostrecent.png')

## Pressure Correction: extract barometric coefficient¶

In [ ]:
from scipy import stats

#fig, ax = plt.subplots(figsize=(15,10))
slope, intercept, r_value, pv, se = stats.linregress(merged_df1['delta_pres'], merged_df1['delta_Ch12'])
print(slope, intercept, r_value, pv, se)
baro12=slope

In [ ]:
slope, intercept, r_value, pv, se = stats.linregress(merged_df1['delta_pres'], merged_df1['delta_Ch13'])
print(slope, intercept, r_value, pv, se)
baro13=slope

In [ ]:
slope, intercept, r_value, pv, se = stats.linregress(merged_df1['delta_pres'], merged_df1['delta_Ch23'])
print(slope, intercept, r_value, pv, se)
baro23=slope

# Pressure Correction

In [ ]:
# pressure correction
merged_df1["Po"] = merged_df1["mslp"].mean()
merged_df1["P-Po"] = merged_df1["mslp"]-merged_df1["Po"]
merged_df1["k(P-Po)"]     = merged_df1["P-Po"]*[-1.0*baro12] # coefficient
merged_df1["exp"]         = np.exp(merged_df1["k(P-Po)"])
merged_df1["ICh12"]       = merged_df1["Ch12"]* merged_df1["exp"]
merged_df1                = merged_df1.drop(columns = ["exp","k(P-Po)","k(P-Po)","P-Po","Po"])

In [ ]:
# pressure correction
merged_df1["Po"] = merged_df1["mslp"].mean()
merged_df1["P-Po"] = merged_df1["mslp"]-merged_df1["Po"]
merged_df1["k(P-Po)"]     = merged_df1["P-Po"]*[-1.0*baro13] # coefficient
merged_df1["exp"]         = np.exp(merged_df1["k(P-Po)"])
merged_df1["ICh13"]       = merged_df1["Ch13"]* merged_df1["exp"]
merged_df1 

In [ ]:
merged_df1["Po"] = merged_df1["mslp"].mean()
merged_df1["P-Po"] = merged_df1["mslp"]-merged_df1["Po"]
merged_df1["k(P-Po)"]     = merged_df1["P-Po"]*[-1.0*baro23] # coefficient
merged_df1["exp"]         = np.exp(merged_df1["k(P-Po)"])
merged_df1["ICh23"]       = merged_df1["Ch23"]* merged_df1["exp"]
merged_df1                = merged_df1.drop(columns = ["exp","k(P-Po)","k(P-Po)","P-Po","Po"])
merged_df1

In [ ]:
plt.rcParams["axes.grid"] = True
fig, (ax1) = plt.subplots(1,figsize=(14, 5))
#fig.suptitle('Counts')
#ax1.hist(df2_hourly['counts'], bins=100,histtype='step',label='Counts',color='blue')
ax1.plot(merged_df1["ICh23"],
marker='.', markersize=1, linestyle='-', linewidth=1, color='red', label='Counts')
ax1.plot(merged_df1["Ch23"],
marker='.', markersize=1, linestyle='-', linewidth=1, color='blue', label='Ch13')
#ax1.set_title('4 Paddle Counts')
#ax1.set_xlim([pd.Timestamp('2020-11-03'), pd.Timestamp('2021-04-20')])
#ax1.legend();

In [ ]:
merged_df1["ICh12_pct"] = ((merged_df1['ICh12'] - merged_df1['ICh12'].mean())/ 
                      merged_df1['ICh12'].mean()*100)
merged_df1["ICh12_m_avg"] = merged_df1.rolling('6H').ICh12_pct.mean()
merged_df1["ICh13_pct"] = ((merged_df1['ICh13'] - merged_df1['ICh13'].mean())/ 
                      merged_df1['ICh13'].mean()*100)
merged_df1["ICh13_m_avg"] = merged_df1.rolling('6H').ICh13_pct.mean()
merged_df1["ICh23_pct"] = ((merged_df1['ICh23'] - merged_df1['ICh23'].mean())/ 
                      merged_df1['ICh23'].mean()*100)
merged_df1["ICh23_m_avg"] = merged_df1.rolling('6H').ICh23_pct.mean()

In [ ]:
plt.rcParams["axes.grid"] = True
fig, (ax1) = plt.subplots(1,figsize=(14, 5))
#fig.suptitle('Counts')
#ax1.hist(df2_hourly['counts'], bins=100,histtype='step',label='Counts',color='blue')
ax1.plot(merged_df1["Ch12_pct"],
marker='.', markersize=1, linestyle='-', linewidth=1, color='red', label='Counts')
ax1.plot(merged_df1["ICh12_pct"],
marker='.', markersize=1, linestyle='-', linewidth=1, color='blue', label='Ch13')
#ax1.set_title('4 Paddle Counts')
#ax1.set_xlim([pd.Timestamp('2020-11-03'), pd.Timestamp('2021-04-20')])
ax1.legend();

In [ ]:
#title1 = now.year()
plt.rcParams["axes.grid"] = True
fig = plt.figure(figsize=(14, 10))
gs = fig.add_gridspec(3)
axs = gs.subplots(sharex=False, sharey=False)
#fig.suptitle('Flux %age Change (Muon001 Detector)')
#axs[0].set_title("Last 24 hours (yesterday:now)")
axs[0].set_title('Last 24 hours:'+str(yesterday)+' to ' +str(now))
axs[0].plot(merged_df1.loc[yesterday:now, 'ICh12_m_avg'],
markerfacecolor='white',markeredgecolor="red", marker='o', markersize=4, linestyle='', linewidth=0.5, color='red', label='Ch12 %')
axs[0].plot(merged_df1.loc[yesterday:now,"ICh13_m_avg"],
marker='*', markersize=4, linestyle='', linewidth=0.5, color='green', label="Ch13 %")
axs[0].plot(merged_df1.loc[yesterday:now,"ICh23_m_avg"],
marker='v', markersize=4, linestyle='', linewidth=0.5, color='blue', label="Ch23 %")

axs[1].set_title("Last 30 days")
axs[1].plot(merged_df1.loc[lastmonth:now, 'ICh12_m_avg'],
markerfacecolor='white',markeredgecolor="red", marker='o', markersize=3, linestyle='', linewidth=0.5, color='red', label='Ch12 %')
axs[1].plot(merged_df1.loc[lastmonth:now,"ICh13_m_avg"],
marker='*', markersize=3, linestyle='', linewidth=0.5, color='green', label="Ch13 %")
axs[1].plot(merged_df1.loc[lastmonth:now,"ICh23_m_avg"],
marker='v', markersize=3, linestyle='', linewidth=0.5, color='blue', label="Ch23 %")

axs[2].set_title("All data")
axs[2].plot(merged_df1["ICh12_m_avg"],
markerfacecolor='white',markeredgecolor="red", marker='o', markersize=3, linestyle='', linewidth=0.5, color='red', label='Ch12 %')
axs[2].plot(merged_df1["ICh13_m_avg"],
marker='*', markersize=3, linestyle='', linewidth=0.5, color='green', label="Ch13 %")
axs[2].plot(merged_df1["ICh23_m_avg"],        
marker='v', markersize=3, linestyle='', linewidth=0.5, color='blue', label="Ch23 %")

#axs[2].plot(df1_hourly["Ch23_m_avg"],
#marker='.', markersize=1, linestyle='-', linewidth=1, color='black', label='Ch23_m_avg')

axs[0].set_ylabel('Flux %age Change')
axs[0].set_ylim([0, 25])
axs[0].legend()
axs[1].set_ylabel('Flux %age Change')
axs[1].set_ylim([-10, 30])
axs[1].legend()
axs[2].set_ylabel('Flux %age Change')
axs[2].set_ylim([-10, 30])
axs[2].legend()
# Hide x labels and tick labels for all but bottom plot.
#for ax in axs:
 #   ax.label_outer()
plt.savefig('/home/cosmic/onlineMon/images/M000_pct_Ch_Pcorr.png')